# Mining Association Rules
In this week's lab, we are going to mine association rules using the Python library `mlxtend`. You can install the library using pip

In [15]:
!pip install mlxtend  

## Hands-On
Before starting to code let's practice with a toy example.
Calculate support and confidence for the following association rules given the shopping receipts database below.
<br><img src="arules.png" width="300" style="float: left"><br>


Answer:
```
{Water} => {Juice}
    Support = 2/4 = 50%
    Confidence = 2/2 = 100%
    
{Juice} => {Water}
    Support = 2/4 = 50%
    Confidence = 2/4 = 50%
    
{Milk} => {Bread}
    Support = 1/4 = 25%
    Confidence = 1/1 = 100%
    
{Juice, Beer} => {Water}
    Support = 1/4 = 25%
    Confidence = 1/2 = 50%
```

Suppose that we have a support threshold of 40% and confidence threshold of 75%. Which rules are most interesting? Why?<br>

Do you think using only the support and confidence measures is enough to identify a rule as intersing?

## Apriori algorithm

We will use the apriori algorithm to mine the frequent itemsets. The `mlxtend` library has an implementation of this algorithm.

In [16]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

## Load data
The dataset we are going to use is a synthetic dataset. It contains the purchases of customers. You can find the source of the dataset [here](https://gist.github.com/Harsh-Git-Hub/2979ec48043928ad9033d8469928e751)

In [17]:
df = pd.read_csv('https://gist.githubusercontent.com/Harsh-Git-Hub/2979ec48043928ad9033d8469928e751/raw/72de943e040b8bd0d087624b154d41b2ba9d9b60/retail_dataset.csv', sep=',')
df.head()

,0,1,2,3,4,5,6
0,Bread,Wine,Eggs,Meat,Cheese,Pencil,Diaper
1,Bread,Cheese,Meat,Diaper,Wine,Milk,Pencil
2,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
3,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
4,Meat,Pencil,Wine,NaN,NaN,NaN,NaN


Each row of the dataset represents items that were purchased together on the same day at the same store.The dataset is a sparse dataset as relatively high percentage of data is NA or NaN or equivalent.
These NaNs make it hard to read the table. Let’s find out how many unique items are actually there in the table.

In [18]:
items = (df['0'].unique())
items

array(['Bread', 'Cheese', 'Meat', 'Eggs', 'Wine', 'Bagel', 'Pencil',
       'Diaper', 'Milk'], dtype=object)

## Data Preprocessing
To make use of the apriori module given by `mlxtend` library, we need to convert the dataset according to it’s liking. apriori module requires a dataframe that has either 0 and 1 or True and False as data. The data we have is all string (name of items), we need to One Hot Encode the data.

In [29]:
dataset = []
for ind, row in df.iterrows():
    transaction = []
    for item in row: 
        # check if item is NaN
        if item == item:
            transaction.append(item)
    dataset.append(transaction)

Next using the `TransactionEncoder` we can transform the transactions to True or False.

In [31]:
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
df.head()

,Bagel,Bread,Cheese,Diaper,Eggs,Meat,Milk,Pencil,Wine
0,False,True,True,True,True,True,False,True,True
1,False,True,True,True,False,True,True,True,True
2,False,False,True,False,True,True,True,False,True
3,False,False,True,False,True,True,True,False,True
4,False,False,False,False,False,True,False,True,True


## Applying Apriori
Now we use the apriori module from mlxtend library to find the frequent itemsets. Before that, let's look at some parameters of this module:

- `df` : One-Hot-Encoded DataFrame or DataFrame that has 0 and 1 or True and False as values
- `min_support` : Floating point value between 0 and 1 that indicates the minimum support required for an itemset to be selected.
- `use_colnames` : This allows to preserve column names for itemset making it more readable.
- `max_len` : Max length of itemset generated. If not set, all possible lengths are evaluated.



In [40]:
freq_items = apriori(df, min_support=0.2, use_colnames=True)
freq_items.head(10)

,support,itemsets
0,0.425397,[Bagel]
1,0.504762,[Bread]
2,0.501587,[Cheese]
3,0.406349,[Diaper]
4,0.438095,[Eggs]
5,0.476190,[Meat]
6,0.501587,[Milk]
7,0.361905,[Pencil]
8,0.438095,[Wine]
9,0.279365,"[Bagel, Bread]"


## Mining Association Rules
Frequent if-then associations called association rules which consists of an antecedent (if) and a consequent (then) in other words `{antecedent} => {consequent}`. The metric can be set to confidence, lift, support, leverage and conviction. In the example below we use the confidence metric with a threshold of __0.6__. This means that we are selecting the rules with a confidence more than __0.6__.

In [45]:
rules = association_rules(freq_items, metric="confidence", min_threshold=0.6)
rules.head(10)

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Bagel),(Bread),0.425397,0.504762,0.279365,0.656716,1.301042,0.064641,1.442650
1,(Bread),(Bagel),0.504762,0.425397,0.279365,0.553459,1.301042,0.064641,1.286787
2,(Bagel),(Milk),0.425397,0.501587,0.225397,0.529851,1.056348,0.012023,1.060116
3,(Diaper),(Bread),0.406349,0.504762,0.231746,0.570312,1.129864,0.026636,1.152554
4,(Milk),(Bread),0.501587,0.504762,0.279365,0.556962,1.103415,0.026183,1.117823
5,(Bread),(Milk),0.504762,0.501587,0.279365,0.553459,1.103415,0.026183,1.116164
6,(Pencil),(Bread),0.361905,0.504762,0.200000,0.552632,1.094836,0.017324,1.107003
7,(Wine),(Bread),0.438095,0.504762,0.244444,0.557971,1.105414,0.023311,1.120375
8,(Eggs),(Cheese),0.438095,0.501587,0.298413,0.681159,1.358008,0.078670,1.563203
9,(Cheese),(Eggs),0.501587,0.438095,0.298413,0.594937,1.358008,0.078670,1.387202


The `rules` dataframe contains all the association rules that we determined as interesting. What do you think? Are they really interesting? What does the __lift__ metric tells you?

Try to generate the above rules again but now with a smaller threshold for confidence, say $0.4$. What do you think about the rules now?

In [50]:
rules = association_rules(freq_items, metric="confidence", min_threshold=0.4)
rules[rules["lift"] < 1]

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
4,(Cheese),(Bread),0.501587,0.504762,0.238095,0.474684,0.940411,-0.015087,0.942742
5,(Bread),(Cheese),0.504762,0.501587,0.238095,0.471698,0.940411,-0.015087,0.943424
8,(Meat),(Bread),0.476190,0.504762,0.206349,0.433333,0.858491,-0.034014,0.873950
9,(Bread),(Meat),0.504762,0.476190,0.206349,0.408805,0.858491,-0.034014,0.886018
15,(Diaper),(Cheese),0.406349,0.501587,0.200000,0.492188,0.981260,-0.003820,0.981490
37,(Milk),(Wine),0.501587,0.438095,0.219048,0.436709,0.996835,-0.000695,0.997539
38,(Wine),(Milk),0.438095,0.501587,0.219048,0.500000,0.996835,-0.000695,0.996825


## Exercise: your turn!
Let's try with a more real and bigger dataset. Load the `Groceries.csv` file and try to find association rules Using the __confidence__ metric and a support threshold of __0.001__ and confidence threshold of __0.05__. 

Check out ther rules you have found that have "bottled beer" as antecedant. Are all of these rules interesting?

### Load the data
Notice that this is not a proper csv file and there are different number of values in each row. So you have to read the file manually.

In [53]:
with open('Groceries.csv', 'r') as f:
    dataset = []
    for line in f:
        transaction = []
        row = line.rstrip('\n').split(',')
        for item in row:
            transaction.append(item)
        dataset.append(transaction)

In [54]:
# create the one_hot encoded dataframe with TransactionEncoder()


In [1]:
# find the frequent itemsets with  min_support=0.001, max_len=2


# find the association rules with metric='confidence' and min_threshold=0.05


# extract rules with 'bottled beer' as antecedents 


